In [1]:
import ctypes
import numpy as np
import json

In [2]:
a = np.load("triangles_a.npy")
b = np.load("triangles_b.npy")

In [3]:
ifortlib = ctypes.CDLL("./src/fortran/ifort_clipping.dll")
def ifort_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = ifortlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

gfortranlib = ctypes.CDLL("./src/fortran/gfortran_clipping.dll")
def gfortran_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = gfortranlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

In [4]:
area_of_intersection = ifort_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-ifort.json", "w"))
np.save("answers/python-ifort.npy", area_of_intersection(a[:10_000], b[:10_000]))

23.8 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
24.3 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
30.9 µs ± 335 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
126 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.04 ms ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.2 ms ± 59 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
106 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.06 s ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
area_of_intersection = gfortran_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-gfortran.json", "w"))
np.save("answers/python-gfortran.npy", area_of_intersection(a[:10_000], b[:10_000]))

23.7 µs ± 752 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
25.1 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
44.8 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
221 µs ± 2.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.13 ms ± 53.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
20.8 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
210 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.12 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
